In [79]:
import numpy as np

p = 2
K = GF(p)
n = 3
m = 2
# R = PolynomialRing(K, x, n) # defines n variables over K
# R.inject_variables(verbose=False) # makes all the variables ready for use

In [22]:
# class QuadraticForm(SageObject):
# adjusted Gram matrix method that works for any ring of char != 2
def my_Gram_matrix(self):
    R = self.base_ring()
    assert R.characteristic() != 2, "Characteristic of the base ring cannot be 2."
    A = (R(1) / R(2)) * self.matrix()
    n = self.dim()

    Int_flag = True
    for i in range(n):
        for j in range(i, n):
            Int_flag &= A[i, j] in R

    # Return the Gram matrix, or an error
    if Int_flag:
        return MatrixSpace(R, n, n)(A)
    raise TypeError("this form does not have an integral Gram matrix")

QuadraticForm.my_Gram_matrix = my_Gram_matrix

In [23]:
# given a polynomial f, output the matrix A, vector B and scalar C s.t.
# f(x) = x^T Ax + Bx + C
def quadratic_to_matrix(f):
    h = f.homogeneous_components()
    A = QuadraticForm(h.get(2)).my_Gram_matrix()
    B = vector([])
    C = 0

    # try/except but for the poor ppl 
    if h.get(1) != None:
        # we make a vector of the coefficients of each basis element
        B = vector([h.get(1).coefficient(R.gens()[i]) for i in range(n)])

    if h.get(0) != None:
        C = h.get(0)

    return A, B, C

In [24]:
# a function that simulates precomposition of a quadratic form
# f(x) = x^T Ax + Bx + C with a n x n matrix S 
# so it returns f \circ S 
def transform_input(S, A, B, C):
    # this throws an error because S^T A S is a matrix but BS is a vector and you cannot
    # put them in together in one vector
    #
    # it works if you return them separately, not in a vector
    return vector(K, [S.transpose()*A*S, B*S, C])

In [106]:
def random_symmetric_matrix(K, n):
    # get a random matrix
    A = random_matrix(K, n, n)
    # and make it symmetric
    return A + A.transpose()

# returns A, B and C like above
def random_quadratic_poly(K, n):
    return random_symmetric_matrix(K, n), random_vector(K, n), K.random_element()

In [86]:
class VectorOfMatrices:
    def __init__(self, matrices):
        self.matrices = matrices
     
    def __mul__(self, other):
        # https://doc.sagemath.org/html/en/reference/matrices/sage/matrix/matrix_dense.html
        if not isinstance(other, sage.matrix.matrix_dense.Matrix_dense):
            raise TypeError("We only multiply by a matrix my dear")

        if other.ncols() != len(self.matrices):
            raise ValueError("Matrix dimensions do not match")

        # multiply them by hand
        result = [sum(other[i, j] * self.matrices[j] for j in range(len(self.matrices))) for i in range(other.nrows())]
        # gives back a list
        return result

    def __repr__(self):
        return f"VectorOfMatrices(\n{self.matrices})"
    
    __rmul__ = __mul__

In [49]:
# an actual example
# instead of generating an actual polynomial first 
# and then getting the corresponding matrices,
# generate the matrices randomly

# here are the actual transformations
T = matrix(GL(m, K).random_element())
S = matrix(GL(n, K).random_element())

# We define the ring with enough variables for both our matrices
R2  = PolynomialRing(K, n^2 + m^2, 'y')
R2.inject_variables(verbose=False)

# first n^2 elements of R2
S_ = matrix(R2, n, n, R2.gens()[:n^2])

# we get the last m^2 variables of R2
T_ = matrix(R2, m, m, R2.gens()[-m^2:])
# print(S_, T_)

# generate whole ass m-sized vector of (homogenous) polynomials
F = []
for i in range(m):
    A = random_symmetric_matrix(K, n)
    F.append(A)

In [206]:
G = [S.transpose()*A*S for A in F]
G_ = [S_.transpose()*A*S_ for A in F]

v = VectorOfMatrices(G)
v_ = VectorOfMatrices(G_)

P = T*v
P_ = T_*v_

system = []
for i in range(m):
    system.append(P_[i] - P[i])
    
flat_system = [A.list() for A in system][0]
# flat_system.append()

# print(system)
# print(flat_system)
# I = R2.ideal(flat_system)
# J = I.groebner_basis()

print(I.dimension())
# print(G[0])
print(S)


1
[43 98 92 49 58]
[20 58 47 22 19]
[29 63 79 75 39]
[45 85 19 28 86]
[67 55 47 28 96]


In [262]:
B = matrix(K, m, n)
for i in range(m):
    B[i] = vector(K, np.random.randint(p, size=n))
    
print(B)

[32 16  2]
[28 58 40]


In [281]:
# print(B[0]*S_)

P = B*S
D = P - B*S_
I = R2.ideal(D.list())
print(D.list())
# J = I.primary_decomposition(); J
# J = R2.ideal(J)
J = I.gens()
J = R2.ideal(J)
J.dimension()

[-32*y0 - 16*y3 - 2*y6 + 19, -32*y1 - 16*y4 - 2*y7 + 44, -32*y2 - 16*y5 - 2*y8 + 13, -28*y0 + 43*y3 - 40*y6 + 50, -28*y1 + 43*y4 - 40*y7 - 16, -28*y2 + 43*y5 - 40*y8 + 38]


7

In [83]:
S1 = matrix(GL(n, K).random_element())
S2 = matrix(GL(n, K).random_element())

R2  = PolynomialRing(K, 2*n^2, 'x')
R2.inject_variables(verbose=False)

# first n^2 elements of R2
S1_ = matrix(R2, n, n, R2.gens()[:n^2])
S2_ = matrix(R2, n, n, R2.gens()[-n^2:])

system = []
for i in range(m):
    A = random_symmetric_matrix(K, n)
    system += (S1*A*S2 - S1_*A*S2_).list()

# system = (D1 - D1_).list() + (D2 - D2_).list() + (D3 - D3_).list()

I = R2.ideal(system)
# J = R2.ideal([x2^2 + x3^3, 2*x2^4])
# J = I.groebner_basis()
print(J.dimension())

17


In [107]:
q = 2
n = 4
K = GF(q)  # Finite field with q elements

# Boolean polynomial ring with 2*n^2 variables in grevlex order
R = PolynomialRing(K, 2 * n**2, names='a', order='degrevlex')

# Generate variable names
# vars_a = [f"a_{i}_{j}" for i in range(1, n+1) for j in range(1, n+1)]
# vars_b = [f"b_{i}_{j}" for i in range(1, n+1) for j in range(1, n+1)]
# vars = vars_a + vars_b
R.inject_variables()  # Inject variables into the namespace

# Create matrices A_ and B_ with the variables
A_ = matrix(R, n, n, [R.gen((i-1)*n + (j-1)) for i in range(1, n+1) for j in range(1, n+1)])
B_ = matrix(R, n, n, [R.gen((i-1)*n + (j-1) + n**2) for i in range(1, n+1) for j in range(1, n+1)])

# Generate random matrices C1 and C2 of full rank
C1 = random_matrix(K, n, n)
while C1.rank() < n:
    C1 = random_matrix(K, n, n)

C2 = random_matrix(K, n, n)
while C2.rank() < n:
    C2 = random_matrix(K, n, n)

# Generate random invertible matrices A and B
A = GL(n, K).random_element()
B = GL(n, K).random_element()

# Compute D1 and D2
D1 = A * C1 * B
D2 = A * C2 * B

# Compute D1_var and D2_var using the symbolic matrices A_ and B_
D1_var = A_ * C1 * B_
D2_var = A_ * C2 * B_

# Create the system of equations
system = []
for i in range(n):
    for j in range(n):
        system.append(D1[i][j] - D1_var[i][j])
        system.append(D2[i][j] - D2_var[i][j])

Defining a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16, a17, a18, a19, a20, a21, a22, a23, a24, a25, a26, a27, a28, a29, a30, a31


In [109]:
# Compute the Groebner basis of the system
I = R.ideal(system)
I.groebner_basis()  # Compute the Groebner basis

# Find the variety (solutions) of the system
# solutions = I.variety()
print(I.dimension())

4
